<a href="https://colab.research.google.com/github/acrossariver/my_gcola/blob/main/%E8%87%AA%E4%BD%9C%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日経２２５スクリーニング


In [4]:
import yfinance as yf
import pandas as pd
import datetime
import requests
import time

# 検索条件
MAX_PER = 15  # 15倍
MAX_PBR = 3.0  # 3.0以下
MIN_UP_CHANGE_RATE = 0.10  # 10%以上の上昇
MIN_DOWN_CHANGE_RATE = -0.05  # 5%以上の下落
MIN_DIVIDEND_YIELD = 2.00  # 2%以上の配当利回り
DAYS = 31 #過去何日から起算しての変化率か？

import pandas as pd

def get_nikkei225_tickers():
    # CSVファイルのURL
    url = "https://raw.githubusercontent.com/nakachan-ing/finance-streamlit/master/nikkei.csv"

    # CSVファイルをpandasで読み込む (shift_jisエンコーディングを指定)
    try:
        df = pd.read_csv(url, encoding='shift_jis')
    except Exception as e:
        print(f"CSVファイルの読み込みエラー: {e}")
        return []

    # データフレームの内容を確認
    #print(df.head())  # 最初の5行を表示

    # 銘柄コードを抽出し、'.T'を追加
    tickers = df['コード'].astype(str).apply(lambda x: x + '.T').tolist()

    return tickers

# 銘柄コードを取得
tickers = get_nikkei225_tickers()

print(f"取得した銘柄数: {len(tickers)}")



# 上昇・下落銘柄を格納するリスト
up_results = []
down_results = []

# スクリーニング処理
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        per = info.get("trailingPE", None)
        pbr = info.get("priceToBook", None)
        dividend_yield = info.get("dividendYield", None)

        if per is None or pbr is None or dividend_yield is None:
            continue

        # 配当利回りとPER、PBRの条件を満たしている銘柄
        if dividend_yield >= MIN_DIVIDEND_YIELD and per <= MAX_PER and pbr <= MAX_PBR:
            # 過去2週間の株価取得
            end = datetime.datetime.today()
            start = end - datetime.timedelta(days=DAYS)
            hist = stock.history(start=start, end=end)

            if len(hist) < 2:
                continue

            start_price = hist['Close'].iloc[0]
            end_price = hist['Close'].iloc[-1]
            change_rate = (end_price - start_price) / start_price

            # 上昇銘柄
            if change_rate >= MIN_UP_CHANGE_RATE:
                up_results.append({
                    "銘柄コード": ticker,
                    "PER": per,
                    "PBR": pbr,
                    "配当利回り": f"{dividend_yield:.2f}%",
                    "株価変化率": f"{change_rate * 100:.2f}%",  # Rename the key to "株価変化率"
                    "上昇/下降": "上昇"
                })
            # 下落銘柄
            elif change_rate <= MIN_DOWN_CHANGE_RATE:
                down_results.append({
                    "銘柄コード": ticker,
                    "PER": per,
                    "PBR": pbr,
                    "配当利回り": f"{dividend_yield:.2f}%",
                    "株価変化率": f"{change_rate * 100:.2f}%",  # Rename the key to "株価変化率"
                    "上昇/下降": "下降"
                })

        # time.sleep(1.5)  # アクセス制限回避のためにスリープ
    except Exception as e:
        print(f"エラー発生: {ticker}, {e}")

#過去何日前から起算して
print(f"過去{DAYS}日前から起算して")

# 銘柄コードと銘柄名の対応表を作成
df_name_map = pd.read_csv("https://raw.githubusercontent.com/nakachan-ing/finance-streamlit/master/nikkei.csv", encoding='shift_jis')
df_name_map['銘柄コード'] = df_name_map['コード'].astype(str) + ".T"
df_name_map = df_name_map[['銘柄コード', '銘柄名']]

# 上昇銘柄のデータフレーム
df_up = pd.DataFrame(up_results)
df_up = pd.merge(df_up, df_name_map, on='銘柄コード', how='left')

# 表示順を調整（銘柄コードと銘柄名を先頭に）
columns = ['銘柄コード', '銘柄名'] + [col for col in df_up.columns if col not in ['銘柄コード', '銘柄名']]
df_up = df_up[columns]

print(f"\n◎{MIN_UP_CHANGE_RATE * 100:.0f}%以上上昇した銘柄数: {len(df_up)}")
df_up = df_up.sort_values(by="株価変化率", ascending=False).reset_index(drop=True)
display(df_up)

# 下落銘柄のデータフレーム
df_down = pd.DataFrame(down_results)
df_down['株価変化率_num'] = df_down['株価変化率'].str.replace('%', '').astype(float)
df_down = df_down.sort_values(by="株価変化率_num").reset_index(drop=True)
df_down = df_down.drop(columns=["株価変化率_num"])

df_down = pd.merge(df_down, df_name_map, on='銘柄コード', how='left')

columns = ['銘柄コード', '銘柄名'] + [col for col in df_down.columns if col not in ['銘柄コード', '銘柄名']]
df_down = df_down[columns]

print(f"\n◎{MIN_DOWN_CHANGE_RATE * 100:.0f}%以上下落した銘柄数: {len(df_down)}")
display(df_down)




取得した銘柄数: 225
過去31日前から起算して

◎10%以上上昇した銘柄数: 3


,銘柄コード,銘柄名,PER,PBR,配当利回り,株価変化率,上昇/下降
0,1803.T,清水建,13.849787,1.229313,2.49%,15.22%,上昇
1,1801.T,大成建,13.369059,1.522885,3.82%,14.75%,上昇
2,1812.T,鹿島,14.224991,1.346475,3.49%,10.89%,上昇



◎-5%以上下落した銘柄数: 44


,銘柄コード,銘柄名,PER,PBR,配当利回り,株価変化率,上昇/下降
0,6506.T,安川電,13.495008,1.772229,2.31%,-21.01%,下降
1,8308.T,りそなＨＤ,11.764707,0.908438,2.12%,-15.62%,下降
2,6724.T,エプソン,11.534279,0.781540,3.67%,-15.55%,下降
3,3099.T,三越伊勢丹,10.416667,1.114271,2.64%,-15.05%,下降
4,5019.T,出光興産,10.183017,0.636526,4.02%,-14.93%,下降
5,4004.T,昭電工,6.337145,0.700748,5.04%,-13.12%,下降
6,8604.T,野村,7.132949,0.674476,5.94%,-12.81%,下降
7,8331.T,千葉銀,13.071582,0.747251,3.60%,-12.54%,下降
8,8411.T,みずほＦＧ,10.080645,0.851589,3.66%,-12.37%,下降
9,8750.T,第一生命ＨＤ,2.440783,0.942491,3.34%,-12.14%,下降


# プライム市場スクリーニング


In [7]:
import yfinance as yf
import pandas as pd
import datetime
import requests
import random
from io import BytesIO

# 検索条件
MAX_PER = 15  # 15倍
MAX_PBR = 3.0  # 3.0以下
MIN_UP_CHANGE_RATE = 0.10  # 10%以上の上昇
MIN_DOWN_CHANGE_RATE = -0.05  # 5%以上の下落
MIN_DIVIDEND_YIELD = 4.00  # 4%以上の配当利回り
DAYS = 31 #過去何日から起算しての変化率か？
PRIME_RANDOM = 333 # ランダムに絞り込むプライム市場の銘柄数

# カスタムヘッダーを定義
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# === JPXのExcelファイルから東証プライム銘柄を取得 ===
xls_url = 'https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls'

# JPXのプライム市場の銘柄をExcelから取得
response = requests.get(xls_url)
xls_file = BytesIO(response.content)

# Excelファイルを読み込む（1枚目のシートを仮定）
df = pd.read_excel(xls_file, sheet_name=0)

# カラム名整形
df.columns = df.columns.str.strip()
df = df.dropna(subset=['コード'])  # コードがNaNの行を除外

# プライム市場に絞り込み
prime_df = df[df['市場・商品区分'].str.contains('プライム', na=False)].copy()

# 銘柄コードを yfinance 形式に
prime_df['銘柄コード'] = prime_df['コード'].astype(str).str.zfill(4) + '.T'

# 銘柄コードを yfinance 形式に（4桁 + ".T"）
tickers = prime_df['コード'].astype(str).str.zfill(4) + '.T'

# ランダムに333銘柄を抽出
tickers_list = random.sample(tickers.tolist(), PRIME_RANDOM)

# 取得銘柄数
print(f"取得銘柄数: {len(tickers_list)}")

# 上昇・下落銘柄を格納するリスト
up_results = []
down_results = []

# スクリーニング処理
for ticker in tickers_list:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        per = info.get("trailingPE", None)
        pbr = info.get("priceToBook", None)
        dividend_yield = info.get("dividendYield", None)

        if per is None or pbr is None or dividend_yield is None:
            continue

        # 配当利回りとPER、PBRの条件を満たしている銘柄
        if per <= MAX_PER and pbr <= MAX_PBR and dividend_yield >= MIN_DIVIDEND_YIELD:
            # 過去2週間の株価取得
            end = datetime.datetime.today()
            start = end - datetime.timedelta(days=DAYS)
            hist = stock.history(start=start, end=end)

            if len(hist) < 2:
                continue

            start_price = hist['Close'].iloc[0]
            end_price = hist['Close'].iloc[-1]
            change_rate = (end_price - start_price) / start_price

            # 上昇銘柄
            if change_rate >= MIN_UP_CHANGE_RATE:
                up_results.append({
                    "銘柄コード": ticker,
                    "PER": per,
                    "PBR": pbr,
                    "配当利回り": f"{dividend_yield:.2f}%",
                    "株価変化率": f"{change_rate * 100:.2f}%",  # Rename the key to "株価変化率"
                    "上昇/下降": "上昇"
                })
            # 下落銘柄
            elif change_rate <= MIN_DOWN_CHANGE_RATE:
                down_results.append({
                    "銘柄コード": ticker,
                    "PER": per,
                    "PBR": pbr,
                    "配当利回り": f"{dividend_yield:.2f}%",
                    "株価変化率": f"{change_rate * 100:.2f}%",  # Rename the key to "株価変化率"
                    "上昇/下降": "下降"
                })

        # time.sleep(1.5)  # アクセス制限回避のためにスリープ
    except Exception as e:
        print(f"エラー発生: {ticker}, {e}")

# 過去何日前から起算して
print(f"過去{DAYS}日前から起算して")

# --- 銘柄コードと銘柄名の対応表を prime_df から取得 ---
prime_df['銘柄コード'] = prime_df['コード'].astype(str).str.zfill(4) + '.T'
df_name_map = prime_df[['銘柄コード', '銘柄名']].copy()

# 上昇銘柄のデータフレーム
df_up = pd.DataFrame(up_results)
df_up = pd.merge(df_up, df_name_map, on='銘柄コード', how='left')

# 表示順を調整（銘柄コードと銘柄名を先頭に）
columns = ['銘柄コード', '銘柄名'] + [col for col in df_up.columns if col not in ['銘柄コード', '銘柄名']]
df_up = df_up[columns]

print(f"\n◎{MIN_UP_CHANGE_RATE * 100:.0f}%以上上昇した銘柄数: {len(df_up)}")
df_up = df_up.sort_values(by="株価変化率", ascending=False).reset_index(drop=True)
display(df_up)

# 下落銘柄のデータフレーム
df_down = pd.DataFrame(down_results)
df_down['株価変化率_num'] = df_down['株価変化率'].str.replace('%', '').astype(float)
df_down = df_down.sort_values(by="株価変化率_num").reset_index(drop=True)
df_down = df_down.drop(columns=["株価変化率_num"])

df_down = pd.merge(df_down, df_name_map, on='銘柄コード', how='left')
columns = ['銘柄コード', '銘柄名'] + [col for col in df_down.columns if col not in ['銘柄コード', '銘柄名']]
df_down = df_down[columns]

print(f"\n◎{MIN_DOWN_CHANGE_RATE * 100:.0f}%以上下落した銘柄数: {len(df_down)}")
display(df_down)


取得銘柄数: 333
エラー発生: 2335.T, Too Many Requests. Rate limited. Try after a while.
エラー発生: 8802.T, Too Many Requests. Rate limited. Try after a while.
エラー発生: 1893.T, Too Many Requests. Rate limited. Try after a while.
エラー発生: 6315.T, Too Many Requests. Rate limited. Try after a while.
エラー発生: 8864.T, Too Many Requests. Rate limited. Try after a while.
エラー発生: 6728.T, Too Many Requests. Rate limited. Try after a while.
エラー発生: 2117.T, Too Many Requests. Rate limited. Try after a while.
エラー発生: 4118.T, Too Many Requests. Rate limited. Try after a while.
エラー発生: 8344.T, Too Many Requests. Rate limited. Try after a while.
エラー発生: 3657.T, Too Many Requests. Rate limited. Try after a while.
エラー発生: 4680.T, Too Many Requests. Rate limited. Try after a while.
エラー発生: 3064.T, Too Many Requests. Rate limited. Try after a while.
エラー発生: 5142.T, Too Many Requests. Rate limited. Try after a while.
エラー発生: 9278.T, Too Many Requests. Rate limited. Try after a while.
エラー発生: 6594.T, Too Many Requests. Rate limited. Try

ERROR:yfinance:401 Client Error: Unauthorized for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/1301.T?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=1301.T&crumb=Edge%3A+Too+Many+Requests


過去31日前から起算して

◎10%以上上昇した銘柄数: 1


,銘柄コード,銘柄名,PER,PBR,配当利回り,株価変化率,上昇/下降
0,1952.T,新日本空調,12.019342,1.430354,4.85%,18.16%,上昇



◎-5%以上下落した銘柄数: 14


,銘柄コード,銘柄名,PER,PBR,配当利回り,株価変化率,上昇/下降
0,5019.T,出光興産,10.183017,0.636526,4.02%,-14.93%,下降
1,8570.T,イオンフィナンシャルサービス,12.936753,0.543159,4.53%,-11.33%,下降
2,6432.T,竹内製作所,8.210225,1.254798,7.78%,-10.90%,下降
3,7201.T,日産自動車,12.049209,0.210100,4.87%,-9.48%,下降
4,5195.T,バンドー化学,11.751315,0.716686,5.08%,-9.16%,下降
5,4041.T,日本曹達,8.431746,0.776121,4.52%,-8.51%,下降
6,9743.T,丹青社,12.675028,1.470378,5.76%,-7.47%,下降
7,4182.T,三菱瓦斯化学,12.562872,0.639124,4.60%,-6.56%,下降
8,6183.T,ベルシステム２４ホールディングス,10.893287,1.235197,5.09%,-6.43%,下降
9,5208.T,有沢製作所,11.764706,0.922160,6.48%,-6.36%,下降
